In [1]:
run_tests = True

from collections import defaultdict
import numpy as np

In [2]:
# Read word list
import re
FIVE_LETTER_WORD = re.compile('[a-z]{5}\n')
with open('words.txt') as wf:
    allwords = [w.strip() for w in wf.readlines() if FIVE_LETTER_WORD.fullmatch(w)]

print(len(allwords), allwords[:20])

2470 ['aback', 'abase', 'abash', 'abate', 'abbas', 'abbey', 'abbot', 'abhor', 'abide', 'abode', 'abort', 'about', 'above', 'abuse', 'abyss', 'acorn', 'acrid', 'actor', 'acute', 'adage']


In [3]:
def score_guess(word, guess):
    word = [c for c in word]
    score = [' '] * 5
    for i in range(5):
        if word[i] == guess[i]:
            score[i] = 'G'
            word[i] = ' '
    for i in range(5):
        if score[i] != ' ': continue
        try:
            word[word.index(guess[i])] = ' '
            score[i] = 'O'
        except ValueError:
            score[i] = 'B'
    return ''.join(score)

if run_tests:
    def test(word, guess, score):
        if score_guess(word, guess) != score:
            raise AssertionError(f'Expected {guess} to match {word} with {score}')
    test("eerie", "geese", "BGOBG")
    test("eerie", "levee", "BGBOG")
    test("eerie", "preen", "BOOOB")
    test("eerie", "yield", "BOOBB")
    test("eerie", "salsa", "BBBBB")
    test("salsa", "slams", "GOOBO")
    test("salsa", "salts", "GGGBO")
    print("OK")

OK


In [4]:
def cache_scores(words):
    l = len(words)
    return [score_guess(words[i], allwords[j]) for i in range(len(words)) for j in range(len(allwords))]

allscores = cache_scores(allwords)

In [12]:
def get_score(scores, w, g):
    return scores[w * len(allwords) + g]

In [5]:
def score_outcomes(outcomes):
    a = np.array(outcomes)
    return (a.mean(), a.std(), a.max())


In [13]:
def compute_guess_outcomes(words, scores):
    """Given a word list, compute the outcome of every guess against every word.

    Returns a dictionary mapping a guess to a list of outcomes, where each outcome
    is a count of words consistent with the guess's score against a particular target.
    """
    outcomes = {}
    for guess in range(len(allwords)):
        o = []
        cache = {}
        for word in range(len(words)):
            score = get_score(scores, word, guess)
            if score not in cache:
                cache[score] = sum(get_score(scores, c, guess) == score for c in range(len(words)))
            o.append(cache[score])
        outcomes[allwords[guess]] = score_outcomes(o)
    return outcomes

In [14]:
try:
    del alloutcomes
except NameError:
    pass
alloutcomes = compute_guess_outcomes(allwords, allscores)

In [15]:
s = sorted(alloutcomes.items(), key=lambda x: x[1][0])
print(" WORD  AVG RMNG  STDDEV  MAX RMNG")
print("------ -------- -------- --------")
for w, (m, sd, mx) in s[:50]:
    print(f"{w}   {m:7.4f}  {sd:7.4f}   {mx:4}")

 WORD  AVG RMNG  STDDEV  MAX RMNG
------ -------- -------- --------
raise   66.7644  50.8820    177
orate   68.9571  57.4488    199
arise   69.9304  57.9477    184
arose   70.5522  57.9881    193
irate   71.0794  61.2721    213
slate   73.4721  70.8099    238
stare   73.6607  69.8385    241
alert   74.2389  62.6779    217
snare   75.1150  68.9029    236
carte   76.3101  72.5381    254
crate   76.7417  74.5508    254
trace   77.6696  74.9467    254
stale   78.3887  70.7871    238
later   78.6113  60.9364    217
least   78.7360  73.8331    238
alter   79.7555  62.4812    217
caret   80.3862  77.0336    254
learn   80.4057  66.2173    233
alone   82.1263  68.4917    211
scare   82.2316  72.7268    238
aisle   82.2453  69.2627    215
atone   83.1482  67.0396    218
saute   83.2211  73.6048    228
carne   83.5960  81.4607    277
caste   84.4640  74.6577    250
store   84.9587  79.4919    274
crane   85.0939  83.7046    277
trail   85.6138  74.2906    254
share   86.5717  78.9706    243
rena

In [18]:
def apply_guess(word, words, scores, guess):
    g = allwords.index(guess)
    w = words.index(word)
    score = get_score(scores, w, g)
    newscores = []
    newwords = []
    for idx, cand in enumerate(words):
        if get_score(scores, idx, g) == score:
            newwords.append(cand)
            newscores.extend(scores[idx * len(allwords):(idx + 1) * len(allwords)])
    return newwords, newscores


In [22]:
def solve_puzzle(word):
    words = allwords
    scores = allscores
    outcomes = alloutcomes
    while True:
        if word not in words:
            raise AssertionException(f"Can't guess {word}; not in list of {len(words)} words")
        if len(words) == 1:
            guess = words[0]
        else:
            best = min(outcomes.items(), key=lambda x: x[1][0])
            guess = best[0]
        yield guess
        if guess == word:
            return
        words, scores = apply_guess(word, words, scores, guess)
        outcomes = compute_guess_outcomes(words, scores)

print(list(solve_puzzle('slump')))
print(list(solve_puzzle('fuzzy')))
print(list(solve_puzzle('gorge')))
print(list(solve_puzzle('witch')))

['raise', 'count', 'aback', 'slump']


['raise', 'mulch', 'fiend', 'fuzzy']


['raise', 'gourd', 'gorge']


['raise', 'lofty', 'debug', 'happy', 'witch']


In [0]:
lengths = {word: len(list(solve_puzzle(word))) for word in allwords}

maxlen = max(lengths.items(), key=lambda x:x[1])[1]
print(maxlen)

print([w for w in lengths if lengths[w] > 6])